In [1]:
import pandas as pd 

In [5]:
rest_df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\restaurant_in_vegas.pickle')
review_df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\vegas_review_reduced.pickle')

In [3]:
rest_df = rest_df[['business_id', 'categories']]
rest_df.head()

,business_id,categories
2,fnZrZlqW1Z8iWgTVDfv_MA,"Mexican, Restaurants, Fast Food"
3,fhNf_sg-XzZ3e7HEVGuOZg,"Burgers, Restaurants, American (Traditional), ..."
10,OWkS1FXNJbozn-qPg3LWxg,"Food, Food Trucks, Restaurants, Pizza"
12,MTx-Zdl_KcU_z9G832XAjg,"Fast Food, Restaurants"
15,Q_dh08clYUPj13GmCRzIVA,"Filipino, Restaurants, Breakfast & Brunch, Foo..."


In [4]:
category_review_df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\vegas_category_review.pickle')

In [15]:
categories = list(category_review_df['category'].unique())

In [16]:
categories.append('business_id')
rest_category_df = pd.DataFrame(index=list(rest_df.index.values), columns=categories)
rest_category_df = rest_category_df.fillna(0)
rest_category_df.head()

,Restaurants,Mexican,Southern,New Mexican Cuisine,Spanish,Tex-Mex,Barbeque,American (Traditional),American (New),Sushi Bars,...,Service Stations,Engraving,Cosmetic Dentists,Dentists,Magicians,Refinishing Services,Tiling,Flooring,Grout Services,business_id
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
rest_category_df['business_id'] = rest_df['business_id']
rest_category_df['categories'] = rest_df['categories']
rest_category_df.head()

,Restaurants,Mexican,Southern,New Mexican Cuisine,Spanish,Tex-Mex,Barbeque,American (Traditional),American (New),Sushi Bars,...,Engraving,Cosmetic Dentists,Dentists,Magicians,Refinishing Services,Tiling,Flooring,Grout Services,business_id,categories
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fnZrZlqW1Z8iWgTVDfv_MA,"Mexican, Restaurants, Fast Food"
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,fhNf_sg-XzZ3e7HEVGuOZg,"Burgers, Restaurants, American (Traditional), ..."
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,OWkS1FXNJbozn-qPg3LWxg,"Food, Food Trucks, Restaurants, Pizza"
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,MTx-Zdl_KcU_z9G832XAjg,"Fast Food, Restaurants"
15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Q_dh08clYUPj13GmCRzIVA,"Filipino, Restaurants, Breakfast & Brunch, Foo..."


In [28]:
for index, row in rest_category_df.iterrows():
    cat = row['categories'].split(", ")
    for c in cat :
        rest_category_df.at[index, c] = 1


In [31]:
rest_category_df = rest_category_df.drop(columns=['Restaurants'])

In [32]:
rest_category_df = rest_category_df.drop(columns=['categories'])

In [33]:
rest_category_df = rest_category_df.astype('category')

In [35]:
rest_category_df.to_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\rest_cat_bool_df.pickle')

In [6]:
rest_expanded_df = pd.DataFrame(columns=['business_id', 'category'])
i=0
for index, row in rest_df.iterrows():
    categories = row['categories'].split(", ")
    for c in categories:
        rest_expanded_df.loc[i] = [row['business_id'], c]
        i+=1

In [11]:
category_review_df = review_df.merge(rest_expanded_df, how='inner', on='business_id')

In [13]:
category_review_df=category_review_df.drop(columns=['business_id'])

In [4]:
rest_category_df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\rest_cat_bool_df.pickle')

In [6]:
user_categories_df = review_df.merge(rest_category_df, how='inner', on='business_id') 

In [8]:
user_categories_df = user_categories_df.drop(columns=['business_id'])

In [11]:
user_categories_df.to_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\user_cat_bool_df.pickle')

In [17]:
import numpy as np

In [18]:
cols = list(user_categories_df.columns)
cols.remove('user_id')
cols.remove('stars')
categorical_data = np.stack([user_categories_df[c].cat.codes.values for c in cols], 1)
categorical_data[:10]

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [22]:
import torch

In [23]:
outputs = ['stars']
outputs = torch.tensor(user_categories_df[outputs].values).flatten()
outputs[:5]

tensor([1, 3, 4, 2, 4])

In [2]:
df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\restaurant_in_vegas.pickle')

In [3]:
writer = open('../data/restaurants/item_index2entity_id.txt', 'w', encoding='utf-8')
entity_cnt = 0
seen_categories = []
for index, row in df.iterrows():
    writer.write('%s\t%d\n' % (row['business_id'], entity_cnt))
    entity_cnt+=1
    for c in row['categories'].split(", "):
        if c not in seen_categories:
            seen_categories.append(c)
            writer.write('%s\t%d\n' % (c, entity_cnt))
            entity_cnt+=1
writer.close()

In [16]:
writer = open('../data/restaurants/kg.txt', 'w', encoding='utf-8')
for index, row in df.iterrows():
    for c in row['categories'].split(", "):
        writer.write('%s\t%s\t%s\n' % (row['business_id'], 'category', c))
writer.close()

In [5]:
df.to_csv('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\restaurants\\ratings.csv', index=False)